<a href="https://colab.research.google.com/github/simonsavine/phasetype/blob/main/matrix_exponential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import scipy as sp
import time

In [19]:
# diagonalizable matrix
DM = np.array([
    [.5, .1, .4],
    [.2, .7, .3],
    [.3, .2, .3]
])

In [20]:
# check is diagonlizable
t0 = time.time()
evals, evecs = np.linalg.eig(DM)
t1 = time.time()
diag_time = t1 - t0
np.linalg.det(evecs)

-0.9615878914705364

In [21]:
# invert eigenvectors
t0 = time.time()
evecs_inv = np.linalg.inv(evecs)
t1 = time.time()
inv_time = t1 - t0

# compute matrix exponential
t0 = time.time()
eDM = np.real(evecs @ np.diag(np.exp(evals)) @ evecs_inv)
t1 = time.time()
exp_time = t1 - t0
total_time = diag_time + inv_time + exp_time
eDM

array([[1.76785882, 0.25568649, 0.64427354],
       [0.45249949, 2.09420083, 0.5820285 ],
       [0.49792352, 0.36839451, 1.49197979]])

In [22]:
# the same, computed with scipy
t0 = time.time()
eDMsp = sp.linalg.expm(DM)
t1 = time.time()
sp_time = t1 - t0
assert np.allclose(eDM, eDMsp)
eDMsp

array([[1.76785882, 0.25568649, 0.64427354],
       [0.45249949, 2.09420083, 0.5820285 ],
       [0.49792352, 0.36839451, 1.49197979]])

In [23]:
# compare times
total_time, sp_time, exp_time

(0.00075531005859375, 0.014783620834350586, 0.00016951560974121094)

In [13]:
# scipy is much slower than the diagonalizable computation

In [14]:
# non-diagonalizable matrix
NDM = np.array([
    [1., 1., 0.],
    [0., 1., 1.],
    [0., 0., 0.]
])

In [15]:
# check non-diagonalizable
evals, evecs = np.linalg.eig(NDM)
np.linalg.det(evecs)

1.2819751242557141e-16

In [16]:
# try compute matrix exponential
eNDM = np.real(evecs @ np.diag(np.exp(evals)) @ np.linalg.inv(evecs))
eNDM

array([[ 2.71828183,  0.        , -1.        ],
       [ 0.        ,  2.71828183,  1.71828183],
       [ 0.        ,  0.        ,  1.        ]])

In [17]:
# the same, computed with scipy
eNDMsp = sp.linalg.expm(NDM)
eNDMsp

array([[2.71828183, 2.71828183, 1.        ],
       [0.        , 2.71828183, 1.71828183],
       [0.        , 0.        , 1.        ]])

In [18]:
# note that they are different, which one is correct?
# apply definition
eNDMdef = np.eye(3)
NDMpow = np.eye(3)
fact = 1
for i in range(1, 20):
  fact *= i
  NDMpow = NDMpow @ NDM
  eNDMdef += NDMpow / fact

eNDMdef

array([[2.71828183, 2.71828183, 1.        ],
       [0.        , 2.71828183, 1.71828183],
       [0.        , 0.        , 1.        ]])

In [38]:
# conclusion: scipy's version is correct, whether the matrix is diagonalizable or not, at the cost of a much slower computation in the diagonalizable case